# Largest Lyapunov exponent calculation

### Algorithm

In [56]:
import numpy as np

In [57]:
# Logistic map
def logistic_map(n=2**15, r=4, x0=.4):
    x = np.zeros(n)
    x[0] = x0
    for i in range(n-1):
        x[i+1] = r*x[i]*(1-x[i])
    return x


# Henon map
def henon_map(n=1000000, a=1.4, b=0.3, x0=.4, y0=0):
    x = np.zeros(n)
    y = np.zeros(n)
    x[0] = x0
    y[0] = y0
    for i in range(1, len(x)):
        x[i] = 1 - a * x[i-1] ** 2 + y[i-1] 
        y[i] = b * x[i-1]
    return x


# Lorenz ts
def lorenz_ts(N=None):
    x = np.array([])
    i = 0
    with open("lorenz.txt") as f:
        for line in f:
            x = np.append(x, float(line))
            i += 1
            if N is not None and i == N:
                break
    return x

sine_data = np.sin(np.arange(0,1000,.01))

lorenz = lorenz_ts()
logistic = logistic_map()
henon = henon_map()

In [58]:
# Use Fast Furie Transform for finding mean period
def find_mean_period(y):
    discrete_fft = np.fft.rfft(y, len(y) * 2 - 1)
    mean_freq = np.sum((np.fft.rfftfreq(len(y) * 2 - 1) * discrete_fft**2)[1:]) / np.sum(discrete_fft[1:]**2)

    return min(int(1.0 / mean_freq.real), int(len(y) / 4))

In [59]:
from sklearn.metrics.pairwise import euclidean_distances

def largest_exponent(series: np.array, J: int, m: int, t: float, 
                     mean_period: int = 1, trajectory_len: int = 20):
    # compute shape values
    N = series.shape[0]
    M = N - (m-1) * J

    # reconstruct with lag algorithm
    x = np.zeros((M, m))
    for i in range(M):
        indexes = np.ones(m) * i + np.arange(m) * J
        x[i] = series[indexes.astype(int)]
    
    # Find nearest neighbor
    distances = euclidean_distances(x)
    neighbor_candidate = np.argsort(distances[:,:M-trajectory_len], axis=1)
    indexes = np.abs(np.arange(M).reshape(-1, 1) - neighbor_candidate) > mean_period
    neighbors = np.array([], dtype=int)
    for index, neighbor in zip(indexes, neighbor_candidate):
        neighbors = np.append(neighbors, neighbor[index][0]) 

    # mean rate of separation
    y = np.zeros(trajectory_len)
    for i in range(trajectory_len):
        neighbors_i = neighbors[:M-i] + i
        neighbors_i[neighbors_i >= M] = (M - 1)
        separation = distances[(np.arange(M - i) + i, neighbors_i)]
        separation = separation[neighbors[:M-i] + i < M]
        separation = separation[separation != 0]
        
        if separation.shape[0] == 0:
            y[i] = np.inf
        else:
            y[i] = np.log(separation).mean() / t

    y = y[np.isfinite(y)]
    slope, _ = np.polyfit(np.arange(1, len(y) + 1), y, 1)
    return slope

In [60]:
mean_period = find_mean_period(logistic[:10000])

print(f"Logistic map {largest_exponent(logistic[:10000], J=1, m=2, t=1, 
                                       trajectory_len=20, mean_period=mean_period):.3f}")
print("Baseline: 0.693")

Logistic map 0.477
Baseline: 0.693


In [61]:
mean_period = find_mean_period(lorenz[:10000])

print(f"Lorenz {largest_exponent(lorenz[:10000], J=1, m=5, t=0.1, trajectory_len=20, mean_period=mean_period):.3f}")
print("Baseline 1.5")

Lorenz 1.146
Baseline 1.5


In [62]:
mean_period = find_mean_period(henon[:10000])

print(f"Henon map {largest_exponent(henon[:10000], J=1, m=2, t=1, 
                                    trajectory_len=20, mean_period=mean_period):.3f}")
print("Baseline 0.418")

Henon map 0.369
Baseline 0.418


In [63]:
mean_period = find_mean_period(sine_data[:10000])

print(f"Sin {largest_exponent(sine_data[:10000], J=3, m=2, t=1, mean_period=mean_period):.3f}")

Sin -0.000
